In [ ]:
import os
import json
import pandas as pd
from experiments.running_experiments import AttributePredictionResult, SampleProcessingResult

In [ ]:
RAW_RESULT_FOLDER_PATH = './results_attributes'
TRANSFORMED_RESULT_FOLDER_PATH = './attribute_prediction_results'

In [ ]:
def parse_sample_data(sample_data: dict) -> tuple[int, list[str]]:
    sample_id = sample_data[SampleProcessingResult.ID]
    final_response_raw = sample_data[SampleProcessingResult.RESULT_RESPONSES][-1]
    predicted_attributes: list[str] = []
    try:
        parsed_response_json = json.loads(final_response_raw[SampleProcessingResult.MODEL_RESPONSE])
        predicted_attributes = parsed_response_json["attributes"]
    except Exception as e:
        pass
    
    return (sample_id, predicted_attributes)

def parse_results(raw_output_list: list[dict]) -> list[AttributePredictionResult]:
    result_list: list[AttributePredictionResult] = []
    for raw_output in raw_output_list:
        sample_id, predicted_attributes = parse_sample_data(raw_output)
        
        prediction_result = AttributePredictionResult(sample_id)
        prediction_result.predicted_attributes = predicted_attributes

        result_list.append(prediction_result)
    return result_list

model_result_dict: dict[str, list[AttributePredictionResult]] = {}

for filename in os.listdir(RAW_RESULT_FOLDER_PATH):
    if not filename.endswith('shot.json'):
        continue
    print(filename)
    file_path = os.path.join(RAW_RESULT_FOLDER_PATH, filename)
    with open(file_path, 'r', encoding='utf-8') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from file {filename}: {e}")
            continue
    prediction_results_list = parse_results(data['results'])
    result_file_path = os.path.join(TRANSFORMED_RESULT_FOLDER_PATH, filename)
    with open(result_file_path, 'w', encoding='utf-8') as f:
        json.dump([prediction.to_dict() for prediction in prediction_results_list], f)
